In [69]:
import subprocess
import random

In [70]:
HARDMAXCAPACITY=2382896

In [71]:
diskBlocks={} #disk=[currentWritingBlock,MaxCapacity]
CURRENTBLOCK=0
MAXCAPACITY=1

command="" #Output
idToBlockDict={} #id=[disk,block,no of blocks]
DISK=0
BLOCK=1
NOOFBLOCKS=2

ID=1
TASK=3
REQUESTTIME=4
DATA=5
READ='1'
WRITE='0'

In [72]:
fOut=open('output.trace','w+')

In [73]:
with open('input.txt') as fIn:
    firstLine = fIn.readline().split()
    for i in range(1,int(firstLine[1])+1):
        diskBlocks[str(i)]=[0,int(firstLine[2])]
    for line in fIn:
        lineList=line.rstrip().split(',')
        if lineList[TASK]=='R':
            if lineList[ID] in idToBlockDict:
                if idToBlockDict[lineList[ID]][DISK]!=lineList[DISK]: #MOVING THE FILE BETWEEN DISKS
                    
                    diskBlocks[idToBlockDict[lineList[ID]][DISK]][MAXCAPACITY]+=int(idToBlockDict[lineList[ID]][NOOFBLOCKS]) #INCREASE MAX CAPACITY
                    if((diskBlocks[lineList[DISK]][CURRENTBLOCK]+int(lineList[NOOFBLOCKS]))<HARDMAXCAPACITY):
                        fOut.write(lineList[REQUESTTIME]+' '
                                   +lineList[DISK]+' '
                                   +str(diskBlocks[lineList[DISK]][CURRENTBLOCK])+' '
                                   +lineList[NOOFBLOCKS]+' '
                                   +WRITE+'\n')  #WRITE TO THE DISK
                    
                        
                    idToBlockDict[lineList[ID]]=[lineList[DISK],
                                                 diskBlocks[lineList[DISK]][CURRENTBLOCK],
                                                 lineList[NOOFBLOCKS]] #UPDATE THE LOCATION OF FILE
                    
                    diskBlocks[lineList[DISK]][CURRENTBLOCK]+=int(lineList[NOOFBLOCKS]) #INCREMENT WRITING BLOCK

                    
                    fOut.write(lineList[REQUESTTIME]+' '
                               +str(idToBlockDict[lineList[ID]][DISK])+' '
                               +str(idToBlockDict[lineList[ID]][BLOCK])+' '
                               +str(idToBlockDict[lineList[ID]][NOOFBLOCKS])+' '
                               +READ+'\n')  #READ FROM THE DISK
                    
                                        
                else:     #JUST READ
                    fOut.write(lineList[REQUESTTIME]+' '
                               +str(idToBlockDict[lineList[ID]][DISK])+' '
                               +str(idToBlockDict[lineList[ID]][BLOCK])+' '
                               +str(idToBlockDict[lineList[ID]][NOOFBLOCKS])+' '
                               +READ+'\n')  #READ FROM THE DISK
                    
            else:   #CREATE RANDOM FILE AND READ
                
                disk=str(random.choice(range(1,len(diskBlocks))))
                
                idToBlockDict[lineList[ID]]=[disk,
                                            diskBlocks[disk][CURRENTBLOCK],
                                            lineList[NOOFBLOCKS]] #ADD THE LOCATION OF FILE
                
                diskBlocks[disk][CURRENTBLOCK]+=int(lineList[NOOFBLOCKS]) #INCREMENT THE DISK BLOCK
                
                
                fOut.write(lineList[REQUESTTIME]+' '
                           +str(idToBlockDict[lineList[ID]][DISK])+' '
                           +str(idToBlockDict[lineList[ID]][BLOCK])+' '
                           +str(idToBlockDict[lineList[ID]][NOOFBLOCKS])+' '
                           +READ+'\n')  #READ FROM THE DISK

            
        elif lineList[TASK]=='W':
            if((diskBlocks[lineList[DISK]][CURRENTBLOCK]+int(lineList[NOOFBLOCKS]))<HARDMAXCAPACITY):           
                fOut.write(lineList[REQUESTTIME]+' '
                                   +lineList[DISK]+' '
                                   +str(diskBlocks[lineList[DISK]][CURRENTBLOCK])+' '
                                   +lineList[NOOFBLOCKS]+' '
                                   +WRITE+'\n')  #WRITE TO THE DISK
            
            if lineList[ID] in idToBlockDict:
                diskBlocks[idToBlockDict[lineList[ID]][DISK]][MAXCAPACITY]+=int(lineList[NOOFBLOCKS])  #INCREASE MAX CAPACITY
            
            idToBlockDict[lineList[ID]]=[lineList[DISK],
                                         diskBlocks[lineList[DISK]][CURRENTBLOCK],
                                         lineList[NOOFBLOCKS]] #UPDATE THE ID
            
            diskBlocks[lineList[DISK]][CURRENTBLOCK]+=int(lineList[NOOFBLOCKS]) #UPDATE THE WRITING BLOCK
                
                
            
fOut.close()    

In [74]:
diskBlocks

{'1': [0, 10000],
 '2': [20, 10010],
 '3': [10, 10000],
 '4': [0, 10000],
 '5': [0, 10000],
 '6': [0, 10000],
 '7': [0, 10000],
 '8': [0, 10000],
 '9': [0, 10000],
 '10': [0, 10000]}

In [75]:
fReturn=open('return.txt','w')
fReturn.write('Virtual disk write stats\n')
with open('input.txt') as fIn:
    firstLine = fIn.readline().split()
    for i in range(1,int(firstLine[1])+1):
        if(diskBlocks[str(i)][CURRENTBLOCK]):
            percent=(diskBlocks[str(i)][CURRENTBLOCK]/int(firstLine[2]))*100
        else:
            percent=0
        fReturn.write('disk'+','+str(i)+','+str(percent)+'\n')

In [76]:
fReturn.write('Actual disk write stats\n')
for i in diskBlocks:
    if(i[CURRENTBLOCK]):
        percent=(diskBlocks[str(i)][CURRENTBLOCK]/HARDMAXCAPACITY)*100
    else:
        percent=0
    fReturn.write('disk'+','+str(i)+','+str(percent)+'\n')
fReturn.close()

## Check if this is working. I did not check

In [77]:
subprocess.run("disksim parms.1B stdout output.trace 0 \"disk1 .. disk"+str(len(diskBlocks))
               +"\" "+"\"Segment size (inblks)\" 64 \"disk*\" \"Scheduler:Scheduling policy\" 4")

FileNotFoundError: [Errno 2] No such file or directory: 'disksim parms.1B stdout output.trace 0 "disk1 .. disk10" "Segment size (inblks)" 64 "disk*" "Scheduler:Scheduling policy" 4': 'disksim parms.1B stdout output.trace 0 "disk1 .. disk10" "Segment size (inblks)" 64 "disk*" "Scheduler:Scheduling policy" 4'